In [1]:
from TestCase import TestCase
import numpy as np
from MonteCarloTreeSearch_Source.build.PyMCTS import MonteCarloTreeSearch

In [2]:
demand_min = 10
demand_max = 400
nmbr_strs = 3
nmbr_prdcts = 4
time_frm = 90
DC_cpcty = 200
truck_capacity = 20
nmbr_brnch_wrldTurn = 10
nmbr_brnch_myTurn = 10
nmbr_simulations_per_rollout = 10
factory_production_limit = 200
exploration_factor = 2
wareHouseStatePy = np.zeros(nmbr_prdcts, dtype=np.int32)
tc = TestCase( nmbr_strs, nmbr_prdcts, DC_capacity = DC_cpcty )
demand_prb_dstrbutnPy = tc.getData(1000, simplex_reduction_factor = 0.3)
    

clipping 0 out of 1080 values


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


0.066.66666666666667
33.333333333333336



In [3]:
tree = MonteCarloTreeSearch(demand_min,demand_max,nmbr_strs, nmbr_prdcts,time_frm,DC_cpcty,truck_capacity,
   nmbr_brnch_wrldTurn, nmbr_brnch_myTurn,nmbr_simulations_per_rollout, factory_production_limit, exploration_factor, wareHouseStatePy, demand_prb_dstrbutnPy)

In [5]:
tree()

RuntimeError: A MyTurn Node is a leaf node